## Adaboost implementation

In this workbook, I will try to implement the Adaboost algorithm, based on the decision tree stump

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [2]:
## Data set. In this example, I will use the Iris dataset from skleanr

iris = datasets.load_iris()

In [3]:
X = iris.data
Y = iris.target

In [4]:
## check other properties

print ("The features' names are: ", iris.feature_names)
print ("The target names are:", iris.target_names)

The features' names are:  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
The target names are: ['setosa' 'versicolor' 'virginica']


In [5]:
## Convert the data to dataframe

iris_df = pd.DataFrame(X, columns=iris.feature_names)
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [6]:
iris_df['target'] = Y
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [7]:
iris_df.iloc[-1,:]

sepal length (cm)    5.9
sepal width (cm)     3.0
petal length (cm)    5.1
petal width (cm)     1.8
target               2.0
Name: 149, dtype: float64

In [8]:
iris_df['target'] = iris_df['target'].apply(lambda x: 1 if x==2 else 0).astype(int)

In [9]:
iris_df0 = iris_df.iloc[0:150, :]

In [10]:
## Here we use iris_df0 as the training dataset

iris_df0['target'].value_counts()

0    100
1     50
Name: target, dtype: int64

#### The Decision tree stump

In [11]:
X0 = iris_df0.iloc[:, 0:4]
Y0 = iris_df0.iloc[:, 4]

In [12]:
print (X0.shape)
print (Y0.shape)

(150, 4)
(150,)


In [27]:
## Train test splitting

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X0, Y0, test_size = 0.3)

In [28]:
## check the size of the X_train and X_test

print (X_train.shape)
assert (X_train.shape[0] == Y_train.shape[0])
print (Y_train.shape)

(105, 4)
(105,)


In [29]:
tree = DecisionTreeClassifier(max_depth=2)

tree.fit(X_train, Y_train)

## prediction
y_pred = tree.predict(X_test)

In [30]:
## use accuracy 

from sklearn.metrics import accuracy_score

print (accuracy_score(y_pred, Y_test))

0.911111111111


In [17]:
logistic = LogisticRegression()
logistic.fit(X_train, Y_train)

y_pred = logistic.predict(X_test)

In [18]:
print (accuracy_score(y_pred, Y_test))

0.933333333333


### 2. Another test data set

In [33]:
X = np.array([0,1,2,3,4,5,6,7,8,9])

X_train = X.reshape(len(X), 1)
Y = np.array([1,1,1,-1,-1,-1,1,1,1,-1])

In [39]:
tree = DecisionTreeClassifier(max_depth=1)

tree.fit(X_train, Y)

## prediction
y_pred = tree.predict(X_train)

In [40]:
print (accuracy_score(y_pred, Y))

0.7


In [42]:
tree.feature_importances_

array([ 1.])

In [43]:
y_pred

array([ 1,  1,  1, -1, -1, -1, -1, -1, -1, -1])

Now let's write down the Adaboost algorithm based on this dataset step by step

**1. Function that split the data set based on the x input**

pseudo code:

input: 1. array x
       2. the number of threholds, N
return: a list of threholds: res&nbsp;

setup 1: res = []<br/>
         min = x.min<br/>
         max = x.max<br/>
         pt = 0<br/>
setup 2:<br/>
         for i in range(N):<br/>
             pt = pt + (max - min)/N<br/>
             res.append(pt)<br/>
return: res<br/>

In [100]:
## implement the above function

def threholds_list(x, n):
    """
    x: input array.
    n: integer, the number of splitting points
    
    return: a list of threholds point
    """
    xmin = np.min(x)
    xmax = np.max(x)
    
    pt = 0; res = []
    for i in range(n):
        pt = pt + np.floor(xmax - xmin - 1)/(n*1.0)
        res.append(pt)
        
    return res

In [89]:
## Using weak classifier for training

def norm(x, threhold, less_than = True):
    if (less_than == True):
        if x <= threhold:
            return 1;
        else:
            return -1;
    else:
        if x > threhold:
            return 1;
        else:
            return -1;

def weakTrain(x, y, threhold, weight):
    """
    input: x, training dataset
           y, target value
           threhold, the threhold from the list that used to
           weight, an array represents the weight for each data
    return: the error rate
    """
    
    error_less = 0;
    error_more = 0;
    
    for ind in range(len(y)):
        if (norm(x[ind,0], threhold, True) != y[ind]):
            error_less = error_less + weight[ind]
        if (norm(x[ind,0], threhold, False) != y[ind]):
            error_more = error_more + weight[ind]
    
    if (error_less <= error_more):
        return threhold, error_less, True
    else:
        return threhold, error_more, False
    

In [90]:
weakTrain(X_train, Y, 3, [1.0/10]*10)

(3, 0.4, True)

In [109]:
## alpha coefficient

def get_alpha(error_rate):
    """
    input: error_rate
    output: the coefficient
    """
    return 0.5*np.log((1-error_rate)/error_rate)

In [110]:
## Implement the algorithm

def fit(x,y, num_iteration):
    """
    This function will train the dataset. 
    input: x, training value
           y, target value
           num_iteration, integer, number of iterations
    return:
    """
    ## length of the data set
    n = y.shape[0]
    
    ## Threholds list
    threholds = threholds_list(x, n)
    
    ## initialize the the weight
    weight = [1.0/n]*n
    
    ## collect all the weak classifier, and 
    weak_class = []
    alphas = []
    
    for it in range(num_iteration):
        
        ## fake minimum, fake threhold
        min_error = np.inf
        min_threhold = 0
        min_less = False
        
        for thre in threholds:
            threhold, error, less = weakTrain(x, y, thre, weight)
            
            if (error <= min_error):
                min_error = error
                min_threhold = threhold
                min_less = less
        
        ## save for latter use
        weak_class.append((min_error, min_threhold, min_less))
        
        ## update weight:
        alpha = get_alpha(min_error)
        alphas.append(alpha)
        
        Zm = 0.0
        for i in range(n):
            Zm = Zm + weight[i]*np.exp(-alpha*y[i]*norm(x[i,0], min_threhold, min_less))
            weight[i] = weight[i]*np.exp(-alpha*y[i]*norm(x[i,0], min_threhold, min_less))
            
        weight = weight/Zm
        
    return alphas, weak_class

In [111]:
## Prediction

def sign(x):
    if (x >= 0):
        return 1
    else:
        return -1

def predict(x,alphas, weak_class):
    """
    This function will make predictions
    """
    y_pred = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        y_pred[i] = sign(np.sum(alphas[k]*norm(x[i], weak_class[k][1], weak_class[k][2]) for k in range(len(alphas))))
    
    return y_pred

In [119]:
## make predictions on the X dataset

alphas, weak_class = fit(X_train,Y, 3)

y_pred = predict(X_train, alphas, weak_class)

accuracy = np.sum(y_pred==Y)*1.0/Y.shape[0]

print ("The accuracy after the adaptive boosting is: %f"%accuracy)

The accuracy after the adaptive boosting is: 0.900000
